In [1]:
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
plt.rcParams['text.usetex'] = True
import scipy
from scipy import integrate as sciint
from scipy import optimize as sciopt
from scipy.special import kn
import numba as nb
import pickle
import funcs

c=2.9792458e10#cm s-1
mev2erg = 1.0218e-6
mpc2 = 938.72*mev2erg # erg
pi = np.pi
day2sec=5184000
Mpc2cm = 3.086e+24
cgi2mJy = 1.e26

T = 1*day2sec
nu = 3e9
n0 = 1.e-3 #cm-3
eps_e = 0.5
eps_b = 0.5
theta_v = 0.25
D_L = 41*Mpc2cm
Gshmax = 100
theta_G = 0.3
theta_c = 0.059
E_g = 1.16e52

funcs.synchrotron_flux(T, nu, n0, eps_e, eps_b, theta_v, D_L, Gshmax, theta_G, theta_c, E_g) * cgi2mJy
#theta=0.01
#phi=0
#a = funcs._flux_integrand(theta, phi, T, nu, n0, eps_e, eps_b, theta_v, Gshmax, theta_G, theta_c, E_g)
#print(a)

/home/cohen/miniconda3/envs/grb/lib/python3.9/site-packages/scipy-1.9.0-py3.9-linux-x86_64.egg/scipy/integrate/_quadpack_py.py:1151: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


KeyboardInterrupt: 

In [ ]:
from math import *
theta=0.01
phi=0

mu = cos(theta)*cos(theta_v) + sin(theta)*sin(theta_v)*cos(phi) 
x = funcs.gaussian_profile(theta, theta_G, theta_c, E_g)/n0
Tarr = np.logspace(-3,2, 100)
tarr=np.zeros_like(Tarr)
bsharr=np.zeros_like(Tarr)
gsharr=np.zeros_like(Tarr)
Rarr=np.zeros_like(Tarr)
for i,T in enumerate(Tarr):
    tarr[i] = funcs.find_emission_time(T*day2sec, x, Gshmax, mu)
    bsharr[i] = sqrt(funcs.B2(tarr[i], x, Gshmax))
    gsharr[i] = sqrt(funcs.G2(tarr[i], x, Gshmax))
    Rarr[i] = funcs._R_over_c(tarr[i], x, Gshmax)*c
fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
#ax.loglog(Tarr,gsharr)
ax.loglog(Tarr,tarr)

In [ ]:
xarr = np.logspace(-4,4,100)
yarr = np.zeros_like(xarr)
yarr2 = np.zeros_like(xarr)
for i,x in enumerate(xarr):
    yarr[i] = funcs._g2(x)
    yarr2[i] = funcs._g(x)
#1 + 4*kn(2,x) / x / (3*kn(3,x)+kn(1,x)-4*kn(2,x))
fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
ax.semilogx(xarr,yarr)
ax.semilogx(xarr, yarr2)
ax.semilogx(xarr, 4/3*np.ones_like(xarr))
ax.semilogx(xarr, 5/3*np.ones_like(xarr))
ax.set_xlabel(r'$x$',fontsize=18)
ax.set_ylabel(r'$\hat{\gamma}^\prime$',fontsize=18)
ax.set_title(r'ratio of specific heats of the shocked fluid (eq.11)',fontsize=18, color='r');

In [ ]:
xarr = np.logspace(-5,5,100)
g_arr = np.zeros_like(xarr)
for i, x in enumerate(xarr):
    g_arr[i] = funcs._G(x)
fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
ax.plot(xarr, g_arr, '.')
ax.plot(xarr, 1+1.5/xarr)
ax.loglog(xarr, 3*xarr**(-1))
#ax.loglog(xarr, 1+0*xarr)
ax.set_xlabel(r'$x$',fontsize=18)
ax.set_ylabel(r'$\Gamma$',fontsize=18)
ax.set_title(r'$\Gamma(x) = \frac{K_3(x)}{K_2(x)}-\frac{1}{x}$ (Lorentz factor of the shocked fluid eq.12)',fontsize=18, color='r');

In [ ]:
gsh_arr = np.logspace(1.e-5,4,1000)
zeta_arr = np.zeros_like(gsh_arr)
for i, gsh in enumerate(gsh_arr):
    zeta_arr[i] = funcs.find_zeta(gsh)
fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
ax.loglog(gsh_arr, zeta_arr)
ax.set_xlabel(r'$\Gamma_{sh}$',fontsize=18)
ax.set_ylabel(r'$x$',fontsize=18)
ax.set_title(r'Values of x by root finding based on eq.5, as a function of $\Gamma_{sh}$',fontsize=18, color='r');


In [ ]:
def e_index(gsh_bsh):
    gsh = np.sqrt(gsh_bsh**2 + 1)
    bsh = np.sqrt(1 - 1/gsh**2)
    zeta = funcs.find_zeta(gsh)
    p = funcs.e_index(zeta, bsh)
    return p

gsh_bsh = np.logspace(-2, 3,100)
parr=np.zeros_like(gsh_bsh)
for i,gb in enumerate(gsh_bsh):
    parr[i] = e_index(gb)

fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
ax.semilogx(gsh_bsh, parr)
ax.semilogx(gsh_bsh, 2*np.ones_like(parr), '--')
ax.semilogx(gsh_bsh, 20/9*np.ones_like(parr), '--')
ax.set_xlabel(r'$\Gamma_{sh}\beta_{sh}$',fontsize=18)
ax.set_ylabel(r'$p$',fontsize=18)
ax.set_title(r'electron index',fontsize=18, color='r');
